# Mongo desde Python 
###  Tratamiento de datos masivos - Rafael Caballero

Para una introducción al uso de Mongo desde Python, ver:

[http://api.mongodb.com/python/current/tutorial.html]

Aquí solo vamos a ver unos pocos ejemplos con dos ficheros json de comentarios de un foro. El primer fichero, usuarios.json, tiene los usuarios, que tienen un identificador y un *nick*. El segundo tiene los comentarios. Nos gustaría añadir el nick a cada comentario, sabiendo que el campo "content.authorId" ya contiene el identificador, lo que nos permitirá cruzar ambos ficheros.

Comenzamos por importar los ficheros de datos. Desde un terminal (con el servidor de Mongo ya arrancado)

```[Python]
mongoimport --db lv --collection usuarios --drop --file usuarios.json
mongoimport --db lv --collection comments --drop --file comments.json
```
Si no se encuentra el comando mongoimport buscarlo y poner el camino completo.

Ya en Python, empezamos por importar la librería que permite cargar datos desde mongo

In [35]:
import pymongo  # la conexión con mongo
from pprint import pprint

In [36]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')

Vamos a recorrer el fichero de usuarios. 
Por cada usuario buscaremos sus comentarios, y a cada comentario le 
añadiremos el *nick* de su autor.

Vamos a ver varias formas de hacerlo. Primero con mucho (demasiado) Python.

Empezamos por acceder a la base de datos. Lo hacemos a través del cliente.

In [37]:
db = client.lv

Ahora accedemos a las colecciones. Vamos a usar 3:
    - La colección de usuarios
    - La colección  de comentarios
    - Una nueva colección, copia de comentarios salvo por que se  ha añadido el nick (para prácticar insert)

In [38]:
usuarios = db.usuarios
comentarios = db.comments
comentarios_nick = db.comments_nick

Como la colección 'comentarios_nick' es de salida borramos su contenido, por si tuviera algo de alguna prueba anterior

In [8]:
comentarios_nick.drop() # Ojo, esto borra completamente la colección


Ya podemos empezar a recorrer la tabla de usuarios. 
Empezamos por echarle un vistazo

In [6]:
user = usuarios.find_one() # el primer usuario
pprint(user)

{'_id': '49dd174eae314bd187ab39d43f76f95e@grupogodo1.fyre.co',
 'display': 'Quisquilloso'}


Ya estamos listos para recorrer todos los usuarios. Por cada uno buscamos sus comentarios y añadimos el nick (clave display)

In [9]:
from timeit import default_timer as timer # para ver cuánto tarda

start = timer()  # a partir e aquí se cuenta el tiempo
inserciones = 0
for user in usuarios.find():
    for comment in comentarios.find({'content.authorId':user['_id']}):
        # añadimos el nick
        # Esto estaría mal en Python
        # comment['content.authorNick'] = user['display']
        comment['content']['authorNick'] = user['display']
        # lo añadimos
        comentarios_nick.insert_one(comment)
        inserciones += 1
        if inserciones % 10000 == 0:
            print(timer()-start)
end = timer()
print(inserciones, ' inserciones en', end - start,' segundos')   

18.062529942070647
30.229725537009436
40.33086654729258
51.34660268512837


KeyboardInterrupt: 

Ufff!! Muy lento. Mejor creamos un índice

In [13]:
result = comentarios.create_index([('content.authorId', pymongo.ASCENDING)],
                                   unique=False)
print(result)

content.authorId_1


In [22]:
from timeit import default_timer as timer # para ver cuánto tarda

comentarios_nick.drop() # borramos lo ya hecho 

start = timer()  # a partir e aquí se cuenta el tiempo
inserciones = 0
for user in usuarios.find():
    for comment in comentarios.find({'content.authorId':user['_id']}):
        # añadimos el nick
        # Esto estaría mal en Python
        # comment['content.authorNick'] = user['display']
        comment['content']['authorNick'] = user['display']
        # lo añadimos
        comentarios_nick.insert_one(comment)
        inserciones += 1
        if inserciones % 10000 == 0:
            print(timer()-start)
end = timer()
print(inserciones, ' inserciones en', end - start,' segundos')   

5.2595282451407
10.033268915852432
14.87313744668404
19.6069694241894
24.38069015284418
29.12248503997921
34.03608584730796
39.16182561645314
44.44135852710747
49.291240753030706
54.28994266426798
59.45225570168918
64.69309870344159
69.75496591995261
75.40213203774897
80.53224885646227
164642  inserciones en 83.13012563031975  segundos


Un poquito mejor, pero mejor dejar más trabajo a Python:
    *update*. Primero vamos a copiar la colección y luego a modificarla con esta orden.

In [51]:
from timeit import default_timer as timer # para ver cuánto tarda

comentarios_nick.drop() # borramos lo ya hecho 

# clonamos la colección
pipeline = [ {"$match": {}}, 
             {"$out": "comments_nick"},
]
comentarios.aggregate(pipeline)

# creamos el índice
result = comentarios_nick.create_index([('content.authorId', pymongo.ASCENDING)],
                                   unique=False)
print(result)


content.authorId_1


In [52]:
start = timer()  # a partir e aquí se cuenta el tiempo
inserciones = 0
for user in usuarios.find():
    result = comentarios_nick.update_many({'content.authorId':user['_id']},
                            {'$set':{'content.authorNick':user['display']}})
    inserciones +=result.matched_count
end = timer()
print(inserciones, ' inserciones en', end - start,' segundos')   

164642  inserciones en 6.32697320853913  segundos


164642  documentos modificados, pero la colección *comments_nick* tiene 204212

**¿Qué ha pasado?**

Lo que ocurre es que la colección contiene documentos que no contienen la clave 'content.authorId':

In [53]:
noAuthor = comentarios_nick.find({'content.authorId':{'$exists':0}}).count()
print('Comentarios sin autor ',noAuthor)                                  

Comentarios sin autor  39570


Veamos alguno de ellos: 

In [47]:
docnoAuthor = comentarios_nick.find_one({'content.authorId':{'$exists':0}})
pprint(docnoAuthor)

{'_id': {'collection': '238492700', 'id': '812132806'},
 'collectionId': '238492700',
 'content': {'createdAt': 1539161274, 'id': '812132806', 'parentId': ''},
 'createdAt': datetime.datetime(2018, 10, 10, 8, 47, 54),
 'downloadAt': datetime.datetime(2018, 10, 11, 5, 1, 26, 798000),
 'erefs': ['2veVx99rDhWZ1mTE2cuqQXqltLjDl2u+7bLLbc6hFh2AJ7zVon/WEY9SPm0Dkz9hS9fMh4RAnj3rWRc='],
 'event': 1539161274543255,
 'source': 5,
 'type': 0,
 'vis': 2}


Observamos que todos los documentos sin autor tienen esta forma. 
Al parecer son documentos internos del foro a los que no vemos utilidad, así que los 
borramos.

In [54]:
result = comentarios_nick.delete_many({'content.authorId':{'$exists':0}})
print('Borrados: ', result.deleted_count)

Borrados:  39570
